In [1]:
#ОБРУЧАЛКА

import pandas as pd

# Загужаем файл остатков и отметает лишнее

ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])

ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')

del ost_obr['Проект']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[['КодСклада','ТоварНо']]
ost_obr1 = ost_obr1[['СерийныйНомер','ТоварНо']]

# Обработка заборников верхние пустые строки в загружаемом файле должны быть удалены

zab = pd.read_excel(r'C:\анаконда\zab.xlsx')
zab = zab[["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','ТоварНо','КодСклада','ЗЛ']
zab.rename(
    columns={"Серийный Но.":'СерийныйНомер', "Товар Но.":'ТоварНо', 
             "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, 
    inplace=True
) 

# Делим файл на две части:необиркованные и без СН на машине

zab_0 = zab[zab.СерийныйНомер.isnull()]
zab_0 = zab_0[['КодСклада','ТоварНо']]
zab = zab[zab.СерийныйНомер.str.contains('СН',na=False)]
zab = zab[['СерийныйНомер','КодСклада']]

# Проставляем магаз в серийники, которые на 4001/3075/опте

df = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = df[['КодСклада','ТоварНо']]

# Соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1,zab_0])

# Догружаем размер/артикул/толщ.плетения и тд

df = pd.read_excel(r'C:\анаконда\карточки.xlsx',sheet_name='обручи',header=6)
result = pd.merge(result,df,left_on=['ТоварНо'],right_on=['Номер'],how='left')
del result['Номер']
result.set_index('КодСклада',drop=True, inplace=True)

# Сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\обручи.xlsx') as writer:
    result.to_excel(writer, sheet_name='остатки')

In [7]:
#БУКВЫ И ЗОДИАКИ

import pandas as pd

# Загужаем файл остатков и отметает лишнее

ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","ТоварноеНаправление","Проект","Тип1"])

# Оставляем только декоративные подвесы и нужные проекты

ost_obr = ost_obr[(ost_obr["Тип1"] == 'ПОДВЕС ДЕКОРАТИВНЫЙ')]
ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del ost_obr['Проект'], ost_obr['Тип1']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[['КодСклада','ТоварНо','ТоварноеНаправление']]
ost_obr1 = ost_obr1[['СерийныйНомер','ТоварНо','ТоварноеНаправление']]

# Обработка заборников верхние пустые строки в загружаемом файле должны быть удалены

zab = pd.read_excel(r'C:\анаконда\zab.xlsx',
                  usecols=["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ","Товарное направление","Тип 1"])

zab = zab[(zab["Тип 1"] == 'ПОДВЕС ДЕКОРАТИВНЫЙ')]
del zab["Тип 1"]
zab = zab.sort_values(by="Номер ЗЛ",ascending = False)

zab.rename(
    columns={"Серийный Но.":'СерийныйНомер', "Товар Но.":'ТоварНо', 
             "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ',"Товарное направление":'ТоварноеНаправление' }, 
    inplace=True
) 

# Делим файл на две части:необиркованные и без СН на машине

zab_0 = zab[zab.СерийныйНомер.isnull()]
zab_0 = zab_0[['КодСклада','ТоварНо','ТоварноеНаправление']]
zab = zab[zab.СерийныйНомер.str.contains('СН',na=False)]
zab = zab[['СерийныйНомер','КодСклада']]

# Проставляем магаз в серийники, которые на 4001/3075/опте

df = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = df[['КодСклада','ТоварНо','ТоварноеНаправление']]

# Соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1,zab_0])


# Догружаем размер/артикул/толщ.плетения и тд

df = pd.read_excel(r'C:\анаконда\карточки.xlsx',sheet_name='буки',header=6)
result = pd.merge(result,df,left_on=['ТоварНо'],right_on=['Номер'],how='left')
del result['Номер']
result.set_index('КодСклада',drop=True, inplace=True)

# Очищаем описание 3 от ненужного мусора
result['Описание 3'] = result['Описание 3'].apply(lambda a: a[:str(a).find(' ')] if str(a).find(' ')>0 else a )


# Сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\буквыизодиаки.xlsx') as writer:
    result.to_excel(writer)
    

In [5]:
# Тип1 артикул размер
import pandas as pd
#загужаем файл остатков и отметает лишнее
ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=["КодСклада", "НазваниеСклада", "Тип1","ПоставщикАрт",
                                "РазмерИзделия","ТоварНо","СерийныйНомер","Проект"])
ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del ost_obr['Проект']

#делим файл на две части:магазы и центральные склады
ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
del ost_obr1['КодСклада']

#обработка заборников верхние пустые строки в загружаемом файле должны быть удалены
zab = pd.read_excel(r'C:\анаконда\zab.xlsx')
zab = zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','КодСклада','ЗЛ']
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

#проставляем магаз в серийники, которые на 4001/3075/опте
ost_obr1 = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = ost_obr1[["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Тип1","ПоставщикАрт","РазмерИзделия"]]

#соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)
del result['НазваниеСклада']

#сохраняем в эксель. нужно доработать сохранение в уже существующий эксель
with pd.ExcelWriter(r'C:\Остатки\остатки.xlsx') as writer:
    result.to_excel(writer)

In [13]:
#ЛЮБОЙ ТОВАР

import pandas as pd
# Загружаем файл остатков и отметает лишнее

ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])

ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del ost_obr['Проект']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[["КодСклада", "ТоварНо", "СерийныйНомер"]]
del ost_obr1['КодСклада']

# обработка заборников верхние пустые строки в загружаемом файле должны быть удалены

zab = pd.read_excel(r'C:\анаконда\zab.xlsx')
zab = zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

# проставляем магаз в серийники, которые на 4001/3075/опте

ost_obr1 = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = ost_obr1[["КодСклада",  "ТоварНо","СерийныйНомер"]]

# соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте

result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)

# сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\остатки.xlsx') as writer:
    result.to_excel(writer)